# 20.0. Data Enrichment - Integrated Dataset - File Analysis

In [1]:
from datetime import datetime
import os
import time
import pandas as pd 
import requests
import urllib3
import json
import sys
import numpy as np
import pandas as pd
from OTXv2 import OTXv2


## Behaviours

In [2]:
behaviours_dir   = "C:\\Users\\leona\\OneDrive\\Desktop\\Tesi\\Pipeline\\Datasets\\Pipeline\\FileAnalysis\\PostDataCleaning\\Integrated_Dataset\\Behaviours.csv"


In [3]:
behaviours_df = pd.read_csv(behaviours_dir, low_memory = False)
behaviours_df

,sha1,sha256,md5,action_name,action_description,maliciousness_level
0,b701b181de8e595225668cb65d5b975e5b300a39,58edde581177c6f96caa4da4cd73396afee160b9152692...,d6cda532ce834b973d5292e05e1d6c38,checks debugger,checks if process is being debugged by a debugger,neutral
1,b701b181de8e595225668cb65d5b975e5b300a39,58edde581177c6f96caa4da4cd73396afee160b9152692...,d6cda532ce834b973d5292e05e1d6c38,generates crypto key,uses windows apis to generate a cryptographic key,neutral
2,b701b181de8e595225668cb65d5b975e5b300a39,58edde581177c6f96caa4da4cd73396afee160b9152692...,d6cda532ce834b973d5292e05e1d6c38,antivm memory available,"checks amount of memory in system, this can be...",neutral
3,b701b181de8e595225668cb65d5b975e5b300a39,58edde581177c6f96caa4da4cd73396afee160b9152692...,d6cda532ce834b973d5292e05e1d6c38,allocates rwx,allocates read-write-execute memory (usually t...,neutral
4,b701b181de8e595225668cb65d5b975e5b300a39,58edde581177c6f96caa4da4cd73396afee160b9152692...,d6cda532ce834b973d5292e05e1d6c38,antivm network adapters,checks adapter addresses which can be used to ...,neutral
...,...,...,...,...,...,...
54514,5d0cf2954a43c613f270ee9c6ea2e74798787725,eaf087b8fe0cf69bed568070ae460d63483525842d4c84...,7b25379caa4340a714b210dfd01fd5f9,asyncrat,NaN,malicious
54515,4b3f7c7b3d8f6748505115fc291b6ac02e1b99a0,c3217157c0ab2d83047f679a073d19d296a03306072ef9...,aff1636c687b809a106aadf5eb0e40ee,contacts a large (345195) amount of remote hosts,NaN,malicious
54516,4b3f7c7b3d8f6748505115fc291b6ac02e1b99a0,c3217157c0ab2d83047f679a073d19d296a03306072ef9...,aff1636c687b809a106aadf5eb0e40ee,creates a large amount of network flows,NaN,malicious
54517,4b3f7c7b3d8f6748505115fc291b6ac02e1b99a0,c3217157c0ab2d83047f679a073d19d296a03306072ef9...,aff1636c687b809a106aadf5eb0e40ee,modifies the watchdog daemon,NaN,malicious


In [4]:
behaviours_categorized_df = behaviours_df.copy()

behaviours_categorized_df['File System'] = 0
behaviours_categorized_df['Virtual Machine'] = 0
behaviours_categorized_df['System Registry'] = 0
behaviours_categorized_df['System Processes'] = 0
behaviours_categorized_df['Network Communications'] = 0
behaviours_categorized_df['Network Activity'] = 0
behaviours_categorized_df['Browser Interactions'] = 0
behaviours_categorized_df['Operating System Activity'] = 0
behaviours_categorized_df['Keylogging Activity'] = 0
behaviours_categorized_df['Data Activity'] = 0
behaviours_categorized_df['Permissions and Privileges'] = 0

behaviours_categorized_df

,sha1,sha256,md5,action_name,action_description,maliciousness_level,File System,Virtual Machine,System Registry,System Processes,Network Communications,Network Activity,Browser Interactions,Operating System Activity,Keylogging Activity,Data Activity,Permissions and Privileges
0,b701b181de8e595225668cb65d5b975e5b300a39,58edde581177c6f96caa4da4cd73396afee160b9152692...,d6cda532ce834b973d5292e05e1d6c38,checks debugger,checks if process is being debugged by a debugger,neutral,0,0,0,0,0,0,0,0,0,0,0
1,b701b181de8e595225668cb65d5b975e5b300a39,58edde581177c6f96caa4da4cd73396afee160b9152692...,d6cda532ce834b973d5292e05e1d6c38,generates crypto key,uses windows apis to generate a cryptographic key,neutral,0,0,0,0,0,0,0,0,0,0,0
2,b701b181de8e595225668cb65d5b975e5b300a39,58edde581177c6f96caa4da4cd73396afee160b9152692...,d6cda532ce834b973d5292e05e1d6c38,antivm memory available,"checks amount of memory in system, this can be...",neutral,0,0,0,0,0,0,0,0,0,0,0
3,b701b181de8e595225668cb65d5b975e5b300a39,58edde581177c6f96caa4da4cd73396afee160b9152692...,d6cda532ce834b973d5292e05e1d6c38,allocates rwx,allocates read-write-execute memory (usually t...,neutral,0,0,0,0,0,0,0,0,0,0,0
4,b701b181de8e595225668cb65d5b975e5b300a39,58edde581177c6f96caa4da4cd73396afee160b9152692...,d6cda532ce834b973d5292e05e1d6c38,antivm network adapters,checks adapter addresses which can be used to ...,neutral,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54514,5d0cf2954a43c613f270ee9c6ea2e74798787725,eaf087b8fe0cf69bed568070ae460d63483525842d4c84...,7b25379caa4340a714b210dfd01fd5f9,asyncrat,NaN,malicious,0,0,0,0,0,0,0,0,0,0,0
54515,4b3f7c7b3d8f6748505115fc291b6ac02e1b99a0,c3217157c0ab2d83047f679a073d19d296a03306072ef9...,aff1636c687b809a106aadf5eb0e40ee,contacts a large (345195) amount of remote hosts,NaN,malicious,0,0,0,0,0,0,0,0,0,0,0
54516,4b3f7c7b3d8f6748505115fc291b6ac02e1b99a0,c3217157c0ab2d83047f679a073d19d296a03306072ef9...,aff1636c687b809a106aadf5eb0e40ee,creates a large amount of network flows,NaN,malicious,0,0,0,0,0,0,0,0,0,0,0
54517,4b3f7c7b3d8f6748505115fc291b6ac02e1b99a0,c3217157c0ab2d83047f679a073d19d296a03306072ef9...,aff1636c687b809a106aadf5eb0e40ee,modifies the watchdog daemon,NaN,malicious,0,0,0,0,0,0,0,0,0,0,0


In [5]:

file_system_keys               = [ 'creat', 'mod', 'delet', 'encrypt', 'unauthor', 'access', 'hidden', 'file', 'system', 'chang', 'name' ]
virtual_machine_keys           = [ 'vm', 'memory', 'control', 'vitual', 'machine']
system_registry_keys           = [ 'registry', 'key', 'settings', 'permissions']
system_process_keys            = [ 'privileg', 'code', 'inject', 'end', 'process', 'termin', 'modif', 'debug']
network_communication_keys     = [ 'data', 'transmiss', 'remote', 'host', 'connect', 'commun', 'protocol', 'address', 'ip', 'port', 'icmp', 'dns']
network_activity_keys          = [ 'traffic', 'monitor', 'scan', 'network', 'detect', 'device', 'send', 'invit', 'receiv']
browser_interactions_keys      = [ 'browser', 'setting', 'script', 'man', 'attack', 'setting', 'privacy']
os_activity_keys               = [ 'os', 'system', 'operating', 'setting', 'disabl', 'permiss', 'modif', 'servic', 'command', 'cmd']
keylogging_keys                = [ 'captur', 'keyboard', 'log', 'sensitiv', 'info', 'data', 'key', 'spy', 'steal', 'runtime', 'read']
data_activity_keys             = [ 'data', 'encrypt', 'corrupt', 'exfilit', 'data', 'file', 'ransom', 'steal', 'spy']
permissions_privilage_keys     = [ 'escal', 'permiss', 'unauthor', 'access', 'modif', 'privac']


In [6]:


def verifica_presenza(parole_chiave, stringa):
    count = 0
    
    for parola_chiave in parole_chiave:
        if parola_chiave.lower() in stringa.lower():
            count = count + 1
            
    return count


for i in range(len(behaviours_categorized_df)):
    
    tag_name = str( behaviours_categorized_df.at[i, 'action_name'] ) + ' ' + str( behaviours_categorized_df.at[i, 'action_description'] )
    
    behaviours_categorized_df.loc[i, 'File System']                = verifica_presenza(file_system_keys, tag_name)
    behaviours_categorized_df.loc[i, 'Virtual Machine']            = verifica_presenza(virtual_machine_keys, tag_name)
    behaviours_categorized_df.loc[i, 'System Registry']            = verifica_presenza(system_registry_keys, tag_name)
    behaviours_categorized_df.loc[i, 'System Processes']           = verifica_presenza(system_process_keys, tag_name)
    behaviours_categorized_df.loc[i, 'Network Communications']     = verifica_presenza(network_communication_keys, tag_name)
    behaviours_categorized_df.loc[i, 'Network Activity']           = verifica_presenza(network_activity_keys, tag_name)
    behaviours_categorized_df.loc[i, 'Browser Interactions']       = verifica_presenza(browser_interactions_keys, tag_name)
    behaviours_categorized_df.loc[i, 'Keylogging Activity']        = verifica_presenza(keylogging_keys, tag_name)
    behaviours_categorized_df.loc[i, 'Data Activity']              = verifica_presenza(data_activity_keys, tag_name)
    behaviours_categorized_df.loc[i, 'Permissions and Privileges'] = verifica_presenza(permissions_privilage_keys, tag_name)
        
behaviours_categorized_df       


,sha1,sha256,md5,action_name,action_description,maliciousness_level,File System,Virtual Machine,System Registry,System Processes,Network Communications,Network Activity,Browser Interactions,Operating System Activity,Keylogging Activity,Data Activity,Permissions and Privileges
0,b701b181de8e595225668cb65d5b975e5b300a39,58edde581177c6f96caa4da4cd73396afee160b9152692...,d6cda532ce834b973d5292e05e1d6c38,checks debugger,checks if process is being debugged by a debugger,neutral,0,0,0,2,0,0,0,0,0,0,0
1,b701b181de8e595225668cb65d5b975e5b300a39,58edde581177c6f96caa4da4cd73396afee160b9152692...,d6cda532ce834b973d5292e05e1d6c38,generates crypto key,uses windows apis to generate a cryptographic key,neutral,0,0,1,0,0,0,0,0,1,0,0
2,b701b181de8e595225668cb65d5b975e5b300a39,58edde581177c6f96caa4da4cd73396afee160b9152692...,d6cda532ce834b973d5292e05e1d6c38,antivm memory available,"checks amount of memory in system, this can be...",neutral,1,3,0,0,0,1,0,0,0,0,0
3,b701b181de8e595225668cb65d5b975e5b300a39,58edde581177c6f96caa4da4cd73396afee160b9152692...,d6cda532ce834b973d5292e05e1d6c38,allocates rwx,allocates read-write-execute memory (usually t...,neutral,0,1,0,0,0,0,0,0,1,0,0
4,b701b181de8e595225668cb65d5b975e5b300a39,58edde581177c6f96caa4da4cd73396afee160b9152692...,d6cda532ce834b973d5292e05e1d6c38,antivm network adapters,checks adapter addresses which can be used to ...,neutral,0,1,0,0,1,2,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54514,5d0cf2954a43c613f270ee9c6ea2e74798787725,eaf087b8fe0cf69bed568070ae460d63483525842d4c84...,7b25379caa4340a714b210dfd01fd5f9,asyncrat,NaN,malicious,0,0,0,0,0,0,0,0,0,0,0
54515,4b3f7c7b3d8f6748505115fc291b6ac02e1b99a0,c3217157c0ab2d83047f679a073d19d296a03306072ef9...,aff1636c687b809a106aadf5eb0e40ee,contacts a large (345195) amount of remote hosts,NaN,malicious,0,0,0,0,2,0,0,0,0,0,0
54516,4b3f7c7b3d8f6748505115fc291b6ac02e1b99a0,c3217157c0ab2d83047f679a073d19d296a03306072ef9...,aff1636c687b809a106aadf5eb0e40ee,creates a large amount of network flows,NaN,malicious,1,0,0,0,0,1,0,0,0,0,0
54517,4b3f7c7b3d8f6748505115fc291b6ac02e1b99a0,c3217157c0ab2d83047f679a073d19d296a03306072ef9...,aff1636c687b809a106aadf5eb0e40ee,modifies the watchdog daemon,NaN,malicious,1,0,0,1,0,0,0,0,0,0,1


In [7]:
behaviours_categorized_df.columns

Index(['sha1', 'sha256', 'md5', 'action_name', 'action_description',
       'maliciousness_level', 'File System', 'Virtual Machine',
       'System Registry', 'System Processes', 'Network Communications',
       'Network Activity', 'Browser Interactions', 'Operating System Activity',
       'Keylogging Activity', 'Data Activity', 'Permissions and Privileges'],
      dtype='object')

In [8]:
grouped_df = behaviours_categorized_df.groupby(['sha1', 'sha256', 'md5', 'action_name', 'action_description', 'maliciousness_level']).sum()
max_column = grouped_df.idxmax(axis=1)

# Se la somma è zero, assegna 'unknown'
max_column[grouped_df.sum(axis=1) == 0] = 'unknown'
behaviours_categorized_df = pd.DataFrame(max_column).reset_index()
behaviours_categorized_df = behaviours_categorized_df.rename(columns = {0 : 'area'})
behaviours_categorized_df

,sha1,sha256,md5,action_name,action_description,maliciousness_level,area
0,007416f9ba10d2e2951a9dcd104b7206ad20d433,094b57a07096fb1f866a104c0b14c84aa99815b6135877...,7baa4836bd81113c16960501f679a81b,antidebug guardpages,guard pages use detected - possible anti-debug...,neutral,System Processes
1,007416f9ba10d2e2951a9dcd104b7206ad20d433,094b57a07096fb1f866a104c0b14c84aa99815b6135877...,7baa4836bd81113c16960501f679a81b,antidebug setunhandledexceptionfilter,setunhandledexceptionfilter detected (possible...,neutral,System Processes
2,007416f9ba10d2e2951a9dcd104b7206ad20d433,094b57a07096fb1f866a104c0b14c84aa99815b6135877...,7baa4836bd81113c16960501f679a81b,cape detected threat,cape detected the asyncrat malware,suspicious,Network Activity
3,007416f9ba10d2e2951a9dcd104b7206ad20d433,094b57a07096fb1f866a104c0b14c84aa99815b6135877...,7baa4836bd81113c16960501f679a81b,cmdline terminate,executed a command line with /c or /r argument...,neutral,System Processes
4,007416f9ba10d2e2951a9dcd104b7206ad20d433,094b57a07096fb1f866a104c0b14c84aa99815b6135877...,7baa4836bd81113c16960501f679a81b,dead connect,attempts to connect to a dead ip:port (1 uniqu...,neutral,Network Communications
...,...,...,...,...,...,...,...
44962,fffd440b2fb9e39598a3a1b533ab73b9f6015480,eb965d5e26d9c037a82b60bc4b9668b8e0a57f5e8a0247...,096492414fe97b8cc2740ef65e909f54,antisandbox sleep,a process attempted to delay the analysis task.,neutral,System Processes
44963,fffd440b2fb9e39598a3a1b533ab73b9f6015480,eb965d5e26d9c037a82b60bc4b9668b8e0a57f5e8a0247...,096492414fe97b8cc2740ef65e909f54,dead host,connects to an ip address that is no longer re...,malicious,Network Communications
44964,fffd440b2fb9e39598a3a1b533ab73b9f6015480,eb965d5e26d9c037a82b60bc4b9668b8e0a57f5e8a0247...,096492414fe97b8cc2740ef65e909f54,network icmp,generates some icmp traffic,suspicious,Network Activity
44965,fffd440b2fb9e39598a3a1b533ab73b9f6015480,eb965d5e26d9c037a82b60bc4b9668b8e0a57f5e8a0247...,096492414fe97b8cc2740ef65e909f54,networkdyndns checkip,connects to a dynamic dns domain,neutral,Network Communications


In [9]:
behaviours_categorized_df.to_csv('C:\\Users\\leona\\OneDrive\\Desktop\\Tesi\\Pipeline\\Datasets\\Pipeline\\FileAnalysis\\PostDataEnrichment\\Step2\\Behaviours.csv')

### YaraRules

In [10]:
yaraRules_dir   = "C:\\Users\\leona\\OneDrive\\Desktop\\Tesi\\Pipeline\\Datasets\\Pipeline\\FileAnalysis\\PostDataCleaning\\Integrated_Dataset\\Yara_Rules.csv"

In [11]:
yaraRules_df = pd.read_csv(yaraRules_dir, low_memory = False)
yaraRules_df

,sha1,sha256,md5,yara_rule_name,yara_rule_description,yara_rule_author
0,1ad9e9761fd6935c0cf5048c9615d0383baac48e,ac901bf5882f14e9e07235b8488b6479b4519addda6dbf...,da9534900ee0d11c9b30cf33152ea03c,iexplorer remcos,detect iexplorer being taken over by remcos,iam-py-test
1,1ad9e9761fd6935c0cf5048c9615d0383baac48e,ac901bf5882f14e9e07235b8488b6479b4519addda6dbf...,da9534900ee0d11c9b30cf33152ea03c,indicator suspicious exe uacbypass cmstpcom,detects windows exceutables bypassing uac usin...,ditekSHen
2,1ad9e9761fd6935c0cf5048c9615d0383baac48e,ac901bf5882f14e9e07235b8488b6479b4519addda6dbf...,da9534900ee0d11c9b30cf33152ea03c,pe imphash,NaN,NaN
3,1ad9e9761fd6935c0cf5048c9615d0383baac48e,ac901bf5882f14e9e07235b8488b6479b4519addda6dbf...,da9534900ee0d11c9b30cf33152ea03c,remcos,remcos payload,kevoreilly
4,1ad9e9761fd6935c0cf5048c9615d0383baac48e,ac901bf5882f14e9e07235b8488b6479b4519addda6dbf...,da9534900ee0d11c9b30cf33152ea03c,remcos rat variants,NaN,NaN
...,...,...,...,...,...,...
3433,5d0cf2954a43c613f270ee9c6ea2e74798787725,eaf087b8fe0cf69bed568070ae460d63483525842d4c84...,7b25379caa4340a714b210dfd01fd5f9,skystars malware imphash,imphash,Skystars LightDefender
3434,5d0cf2954a43c613f270ee9c6ea2e74798787725,eaf087b8fe0cf69bed568070ae460d63483525842d4c84...,7b25379caa4340a714b210dfd01fd5f9,win asyncrat j1,detects asyncrat,Johannes Bader @viql
3435,5d0cf2954a43c613f270ee9c6ea2e74798787725,eaf087b8fe0cf69bed568070ae460d63483525842d4c84...,7b25379caa4340a714b210dfd01fd5f9,win asyncrat w0,detect asyncrat in memory,JPCERT/CC Incident Response Group
3436,4b3f7c7b3d8f6748505115fc291b6ac02e1b99a0,c3217157c0ab2d83047f679a073d19d296a03306072ef9...,aff1636c687b809a106aadf5eb0e40ee,linux generic ipv6 catcher,elf samples using ipv6 addresses,@_lubiedo


In [12]:
behaviours_categorized_df = yaraRules_df.copy()

behaviours_categorized_df['Antivirus Evasion'] = 0
behaviours_categorized_df['Malware Families'] = 0
behaviours_categorized_df['Exploits and Vulnerabilities'] = 0
behaviours_categorized_df['Backdoors and Remote Access Trojans '] = 0
behaviours_categorized_df['Rootkits and Stealth Techniques'] = 0
behaviours_categorized_df['Network Activity'] = 0
behaviours_categorized_df['Browser Interactions'] = 0
behaviours_categorized_df['Operating System Activity'] = 0
behaviours_categorized_df['Keylogging Activity'] = 0
behaviours_categorized_df['Data Activity'] = 0
behaviours_categorized_df['Permissions and Privileges'] = 0

behaviours_categorized_df

,sha1,sha256,md5,yara_rule_name,yara_rule_description,yara_rule_author,Antivirus Evasion,Malware Families,Exploits and Vulnerabilities,Backdoors and Remote Access Trojans,Rootkits and Stealth Techniques,Network Activity,Browser Interactions,Operating System Activity,Keylogging Activity,Data Activity,Permissions and Privileges
0,1ad9e9761fd6935c0cf5048c9615d0383baac48e,ac901bf5882f14e9e07235b8488b6479b4519addda6dbf...,da9534900ee0d11c9b30cf33152ea03c,iexplorer remcos,detect iexplorer being taken over by remcos,iam-py-test,0,0,0,0,0,0,0,0,0,0,0
1,1ad9e9761fd6935c0cf5048c9615d0383baac48e,ac901bf5882f14e9e07235b8488b6479b4519addda6dbf...,da9534900ee0d11c9b30cf33152ea03c,indicator suspicious exe uacbypass cmstpcom,detects windows exceutables bypassing uac usin...,ditekSHen,0,0,0,0,0,0,0,0,0,0,0
2,1ad9e9761fd6935c0cf5048c9615d0383baac48e,ac901bf5882f14e9e07235b8488b6479b4519addda6dbf...,da9534900ee0d11c9b30cf33152ea03c,pe imphash,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0
3,1ad9e9761fd6935c0cf5048c9615d0383baac48e,ac901bf5882f14e9e07235b8488b6479b4519addda6dbf...,da9534900ee0d11c9b30cf33152ea03c,remcos,remcos payload,kevoreilly,0,0,0,0,0,0,0,0,0,0,0
4,1ad9e9761fd6935c0cf5048c9615d0383baac48e,ac901bf5882f14e9e07235b8488b6479b4519addda6dbf...,da9534900ee0d11c9b30cf33152ea03c,remcos rat variants,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3433,5d0cf2954a43c613f270ee9c6ea2e74798787725,eaf087b8fe0cf69bed568070ae460d63483525842d4c84...,7b25379caa4340a714b210dfd01fd5f9,skystars malware imphash,imphash,Skystars LightDefender,0,0,0,0,0,0,0,0,0,0,0
3434,5d0cf2954a43c613f270ee9c6ea2e74798787725,eaf087b8fe0cf69bed568070ae460d63483525842d4c84...,7b25379caa4340a714b210dfd01fd5f9,win asyncrat j1,detects asyncrat,Johannes Bader @viql,0,0,0,0,0,0,0,0,0,0,0
3435,5d0cf2954a43c613f270ee9c6ea2e74798787725,eaf087b8fe0cf69bed568070ae460d63483525842d4c84...,7b25379caa4340a714b210dfd01fd5f9,win asyncrat w0,detect asyncrat in memory,JPCERT/CC Incident Response Group,0,0,0,0,0,0,0,0,0,0,0
3436,4b3f7c7b3d8f6748505115fc291b6ac02e1b99a0,c3217157c0ab2d83047f679a073d19d296a03306072ef9...,aff1636c687b809a106aadf5eb0e40ee,linux generic ipv6 catcher,elf samples using ipv6 addresses,@_lubiedo,0,0,0,0,0,0,0,0,0,0,0


In [13]:

file_system_keys               = [ 'creat', 'mod', 'delet', 'encrypt', 'unauthor', 'access', 'hidden', 'file', 'system', 'chang', 'name' ]
virtual_machine_keys           = [ 'vm', 'memory', 'control', 'vitual', 'machine']
system_registry_keys           = [ 'registry', 'key', 'settings', 'permissions']
system_process_keys            = [ 'privileg', 'code', 'inject', 'end', 'process', 'termin', 'modif', 'debug']
network_communication_keys     = [ 'data', 'transmiss', 'remote', 'host', 'connect', 'commun', 'protocol', 'address', 'ip', 'port', 'icmp', 'dns']
network_activity_keys          = [ 'traffic', 'monitor', 'scan', 'network', 'detect', 'device', 'send', 'invit', 'receiv']
browser_interactions_keys      = [ 'browser', 'setting', 'script', 'man', 'attack', 'setting', 'privacy']
os_activity_keys               = [ 'os', 'system', 'operating', 'setting', 'disabl', 'permiss', 'modif', 'servic', 'command', 'cmd']
keylogging_keys                = [ 'captur', 'keyboard', 'log', 'sensitiv', 'info', 'data', 'key', 'spy', 'steal', 'runtime', 'read']
data_activity_keys             = [ 'data', 'encrypt', 'corrupt', 'exfilit', 'data', 'file', 'ransom', 'steal', 'spy']
permissions_privilage_keys     = [ 'escal', 'permiss', 'unauthor', 'access', 'modif', 'privac']


In [15]:


def verifica_presenza(parole_chiave, stringa):
    count = 0
    
    for parola_chiave in parole_chiave:
        if parola_chiave.lower() in stringa.lower():
            count = count + 1
            
    return count


for i in range(len(behaviours_categorized_df)):
    
    tag_name = str( behaviours_categorized_df.at[i, 'yara_rule_name'] ) + ' ' + str( behaviours_categorized_df.at[i, 'yara_rule_description'] )
    
    behaviours_categorized_df.loc[i, 'File System']                = verifica_presenza(file_system_keys, tag_name)
    behaviours_categorized_df.loc[i, 'Virtual Machine']            = verifica_presenza(virtual_machine_keys, tag_name)
    behaviours_categorized_df.loc[i, 'System Registry']            = verifica_presenza(system_registry_keys, tag_name)
    behaviours_categorized_df.loc[i, 'System Processes']           = verifica_presenza(system_process_keys, tag_name)
    behaviours_categorized_df.loc[i, 'Network Communications']     = verifica_presenza(network_communication_keys, tag_name)
    behaviours_categorized_df.loc[i, 'Network Activity']           = verifica_presenza(network_activity_keys, tag_name)
    behaviours_categorized_df.loc[i, 'Browser Interactions']       = verifica_presenza(browser_interactions_keys, tag_name)
    behaviours_categorized_df.loc[i, 'Keylogging Activity']        = verifica_presenza(keylogging_keys, tag_name)
    behaviours_categorized_df.loc[i, 'Data Activity']              = verifica_presenza(data_activity_keys, tag_name)
    behaviours_categorized_df.loc[i, 'Permissions and Privileges'] = verifica_presenza(permissions_privilage_keys, tag_name)
        
behaviours_categorized_df       


,sha1,sha256,md5,yara_rule_name,yara_rule_description,yara_rule_author,Antivirus Evasion,Malware Families,Exploits and Vulnerabilities,Backdoors and Remote Access Trojans,...,Browser Interactions,Operating System Activity,Keylogging Activity,Data Activity,Permissions and Privileges,File System,Virtual Machine,System Registry,System Processes,Network Communications
0,1ad9e9761fd6935c0cf5048c9615d0383baac48e,ac901bf5882f14e9e07235b8488b6479b4519addda6dbf...,da9534900ee0d11c9b30cf33152ea03c,iexplorer remcos,detect iexplorer being taken over by remcos,iam-py-test,0,0,0,0,...,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0
1,1ad9e9761fd6935c0cf5048c9615d0383baac48e,ac901bf5882f14e9e07235b8488b6479b4519addda6dbf...,da9534900ee0d11c9b30cf33152ea03c,indicator suspicious exe uacbypass cmstpcom,detects windows exceutables bypassing uac usin...,ditekSHen,0,0,0,0,...,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0
2,1ad9e9761fd6935c0cf5048c9615d0383baac48e,ac901bf5882f14e9e07235b8488b6479b4519addda6dbf...,da9534900ee0d11c9b30cf33152ea03c,pe imphash,NaN,NaN,0,0,0,0,...,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0
3,1ad9e9761fd6935c0cf5048c9615d0383baac48e,ac901bf5882f14e9e07235b8488b6479b4519addda6dbf...,da9534900ee0d11c9b30cf33152ea03c,remcos,remcos payload,kevoreilly,0,0,0,0,...,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0
4,1ad9e9761fd6935c0cf5048c9615d0383baac48e,ac901bf5882f14e9e07235b8488b6479b4519addda6dbf...,da9534900ee0d11c9b30cf33152ea03c,remcos rat variants,NaN,NaN,0,0,0,0,...,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3433,5d0cf2954a43c613f270ee9c6ea2e74798787725,eaf087b8fe0cf69bed568070ae460d63483525842d4c84...,7b25379caa4340a714b210dfd01fd5f9,skystars malware imphash,imphash,Skystars LightDefender,0,0,0,0,...,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0
3434,5d0cf2954a43c613f270ee9c6ea2e74798787725,eaf087b8fe0cf69bed568070ae460d63483525842d4c84...,7b25379caa4340a714b210dfd01fd5f9,win asyncrat j1,detects asyncrat,Johannes Bader @viql,0,0,0,0,...,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0
3435,5d0cf2954a43c613f270ee9c6ea2e74798787725,eaf087b8fe0cf69bed568070ae460d63483525842d4c84...,7b25379caa4340a714b210dfd01fd5f9,win asyncrat w0,detect asyncrat in memory,JPCERT/CC Incident Response Group,0,0,0,0,...,0,0,0,0,0,0.0,1.0,0.0,0.0,0.0
3436,4b3f7c7b3d8f6748505115fc291b6ac02e1b99a0,c3217157c0ab2d83047f679a073d19d296a03306072ef9...,aff1636c687b809a106aadf5eb0e40ee,linux generic ipv6 catcher,elf samples using ipv6 addresses,@_lubiedo,0,0,0,0,...,0,0,0,0,0,0.0,0.0,0.0,0.0,2.0


In [16]:
behaviours_categorized_df.columns

Index(['sha1', 'sha256', 'md5', 'yara_rule_name', 'yara_rule_description',
       'yara_rule_author', 'Antivirus Evasion', 'Malware Families',
       'Exploits and Vulnerabilities', 'Backdoors and Remote Access Trojans ',
       'Rootkits and Stealth Techniques', 'Network Activity',
       'Browser Interactions', 'Operating System Activity',
       'Keylogging Activity', 'Data Activity', 'Permissions and Privileges',
       'File System', 'Virtual Machine', 'System Registry', 'System Processes',
       'Network Communications'],
      dtype='object')

In [19]:
grouped_df = behaviours_categorized_df.groupby(['sha1', 'sha256', 'md5', 'yara_rule_name', 'yara_rule_description', 'yara_rule_author']).sum()
max_column = grouped_df.idxmax(axis=1)

# Se la somma è zero, assegna 'unknown'
max_column[grouped_df.sum(axis=1) == 0] = 'unknown'
behaviours_categorized_df = pd.DataFrame(max_column).reset_index()
behaviours_categorized_df = behaviours_categorized_df.rename(columns = {0 : 'area'})
behaviours_categorized_df

,sha1,sha256,md5,yara_rule_name,yara_rule_description,yara_rule_author,area
0,01facbf9132aea6e12300d19119f434a7079f8f0,7c911b4d457b1b6124a1211cd4737c056e9a12dc930a05...,f41a14859f324f4b208163475c347837,asyncrat,detect asyncrat in memory,JPCERT/CC Incident Response Group,Network Activity
1,01facbf9132aea6e12300d19119f434a7079f8f0,7c911b4d457b1b6124a1211cd4737c056e9a12dc930a05...,f41a14859f324f4b208163475c347837,win asyncrat j1,detects asyncrat,Johannes Bader @viql,Network Activity
2,0306b6dcf5035b9e6759e4573f629e4ba7191395,8499e016c470f2735d4873be23619a68ce08a7a1f67b72...,6db93aef69435a63810b7cffec6b30d2,chrome stealer bin mem,chrome in files like avemaria,James_inthe_box,Data Activity
3,0306b6dcf5035b9e6759e4573f629e4ba7191395,8499e016c470f2735d4873be23619a68ce08a7a1f67b72...,6db93aef69435a63810b7cffec6b30d2,indicator suspicious exe uacbypass eventviewer,detects windows exceutables potentially bypass...,ditekSHen,Network Activity
4,0306b6dcf5035b9e6759e4573f629e4ba7191395,8499e016c470f2735d4873be23619a68ce08a7a1f67b72...,6db93aef69435a63810b7cffec6b30d2,keylog bin mem,contains keylog,James_inthe_box,Keylogging Activity
...,...,...,...,...,...,...,...
2382,ff91376da89e90d19e7c9f53dd8915be22e83627,d9dcaa71fb3a6eb36123628d089cfa59f1b3f01120626a...,31b76b0b5ab72c603ef54efa63c32b22,indicator suspicious stomped pecompilation tim...,detect executables with stomped pe compilation...,ditekSHen,Network Activity
2383,ff91376da89e90d19e7c9f53dd8915be22e83627,d9dcaa71fb3a6eb36123628d089cfa59f1b3f01120626a...,31b76b0b5ab72c603ef54efa63c32b22,nanocore,detect nanocore in memory,JPCERT/CC Incident Response Group,Network Activity
2384,ff91376da89e90d19e7c9f53dd8915be22e83627,d9dcaa71fb3a6eb36123628d089cfa59f1b3f01120626a...,31b76b0b5ab72c603ef54efa63c32b22,nanocore rat feb18 1,detects nanocore rat,Florian Roth,Network Activity
2385,ff91376da89e90d19e7c9f53dd8915be22e83627,d9dcaa71fb3a6eb36123628d089cfa59f1b3f01120626a...,31b76b0b5ab72c603ef54efa63c32b22,nanocore rat gen 2,detetcs the nanocore rat,Florian Roth,unknown


In [20]:
behaviours_categorized_df.to_csv('C:\\Users\\leona\\OneDrive\\Desktop\\Tesi\\Pipeline\\Datasets\\Pipeline\\FileAnalysis\\PostDataEnrichment\\Step2\\YaraRules.csv')

### ATTACKS

In [21]:
pip install mitreattack-python


Note: you may need to restart the kernel to use updated packages.


In [22]:
import requests

def get_attack_object(attack_id):
    base_url = "https://attack.mitre.org/api"
    endpoint = f"/v1/objects/{attack_id}"

    # Construct the full URL
    full_url = base_url + endpoint

    try:
        # Make the GET request
        response = requests.get(full_url)

        # Check if the request was successful (status code 200)
        if response.status_code == 200:
            # Parse the JSON response
            attack_object = response.json()
            
            # Do something with the retrieved object
            print("Attack Object:", attack_object)
        else:
            print(f"Error: {response.status_code} - {response.text}")

    except Exception as e:
        print(f"An error occurred: {str(e)}")

# Example: Get object for attack_id "T1234"
get_attack_object("T1234")


Error: 404 - 
<!DOCTYPE html>
<html lang='en'>

<head>
        <script async src="https://www.googletagmanager.com/gtag/js?id=UA-62667723-1"></script>
        <script>
            window.dataLayer = window.dataLayer || [];
            function gtag(){dataLayer.push(arguments);}
            gtag('js', new Date());
            gtag('config', 'UA-62667723-1');
        </script>
        <meta name="google-site-verification" content="2oJKLqNN62z6AOCb0A0IXGtbQuj-lev5YPAHFF_cbHQ"/>

        <meta charset='utf-8'>
        <meta name='viewport' content='width=device-width, initial-scale=1,shrink-to-fit=no'>
        <meta http-equiv="X-UA-Compatible" content="IE=edge">
        <link rel='shortcut icon' href='/theme/favicon.ico' type='image/x-icon'>
        <title>404 | MITRE ATT&CK&reg;</title>
        <!-- Bootstrap CSS -->
        <link rel='stylesheet' href='/theme/style/bootstrap.min.css' />
        <link rel='stylesheet' href='/theme/style/bootstrap-tourist.css' />
        <link rel='styles